In [59]:
%reset -sf

In [121]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
# import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [122]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [107]:
srr = """
""".strip()#.split("\n")

In [108]:
srr = """
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#
""".strip().split("\n")

In [123]:
n = len(srr)
m = len(srr[0])

In [124]:
xrr = [[[[0 for _ in range(3)] for _ in range(2000)] for _ in range(m)] for _ in range(n)]

In [125]:
for i,row in enumerate(srr):
    for j,cell in enumerate(row):
        if cell == "#":
            x,y = i,j
            for z in range(2000):
                for f in range(3):
                    xrr[x][y][z][f] = 1
        if cell == ".":
            continue
        if cell == ">":
            x,y = i,j
            dx,dy = 0,1
            for z in range(2000):
                for f in range(3):
                    xrr[x][y][z][f] = 1
                x, y = x+dx, y+dy
                if srr[x][y] == "#":
                    y = 1
        if cell == "<":
            x,y = i,j
            dx,dy = 0,-1
            for z in range(2000):
                for f in range(3):
                    xrr[x][y][z][f] = 1
                x, y = x+dx, y+dy
                if srr[x][y] == "#":
                    y = m - 2
        if cell == "v":
            x,y = i,j
            dx,dy = 1,0
            for z in range(2000):
                for f in range(3):
                    xrr[x][y][z][f] = 1
                x, y = x+dx, y+dy
                if srr[x][y] == "#":
                    x = 1
        if cell == "^":
            x,y = i,j
            dx,dy = -1,0
            for z in range(2000):
                for f in range(3):
                    xrr[x][y][z][f] = 1
                x, y = x+dx, y+dy
                if srr[x][y] == "#":
                    x = n - 2

In [126]:
import numpy as np

In [127]:
np.array(xrr).shape

(27, 122, 2000, 3)

In [128]:
# for z in range(20):
#     for i in range(n):
#         for j in range(m):
#             print(xrr[i][j][z], end="")
#         print("\n", end="")
#     print()

In [129]:
states = set([(0,1,0,0)])
for tidx in range(2000):
    print(len(states))
    new_states = set()
    for x,y,z,f in states:
        z += 1
        for dx,dy in d4:
            xx = x+dx
            yy = y+dy
            if not (0 <= xx < n):
                continue
            if xrr[xx][yy][z][f] == 1:
                continue
            if xx == n-1 and yy == m-2 and f == 0:
                new_states.add((xx,yy,z,f+1))
            if xx == 0 and yy == 1 and f == 1:
                new_states.add((xx,yy,z,f+1))
            if xx == n-1 and yy == m-2 and f == 2:
                print(tidx)
                raise
            new_states.add((xx,yy,z,f))
        if xrr[x][y][z][f] == 0:
            new_states.add((x,y,z,f))
    states = new_states

1
2
3
5
4
4
7
14
12
15
16
16
19
22
25
23
25
24
28
29
33
34
35
33
39
39
38
39
43
45
49
46
44
42
47
42
53
56
58
57
60
70
84
89
98
102
102
105
118
122
114
116
127
133
122
150
141
151
156
140
143
155
156
161
165
163
174
181
186
195
195
195
203
196
198
198
211
226
213
221
225
222
234
227
229
219
237
235
240
243
253
264
266
263
257
264
283
268
286
301
288
295
312
312
319
335
327
321
346
358
350
347
347
350
346
347
369
363
361
367
329
349
365
369
362
357
375
388
399
390
393
403
412
415
413
426
450
468
470
479
480
476
454
458
493
494
493
492
522
541
505
531
561
528
548
520
546
536
559
550
554
545
556
551
537
548
562
568
564
573
594
610
593
602
596
576
584
615
629
654
586
618
638
628
625
647
670
671
651
623
669
634
674
664
656
688
674
681
675
678
615
659
668
652
660
670
672
686
687
675
672
659
693
668
717
683
722
697
692
698
706
701
720
718
713
677
690
711
708
705
700
693
684
704
708
703
743
745
718
741
661
706
733
751
785
782
808
784
802
812
731
768
791
805
796
825
823
825
840
859
832
848
837


RuntimeError: No active exception to reraise

In [130]:
tidx + 1

856

In [78]:
new_states

{(1, 3, 18), (1, 5, 18), (2, 1, 18), (3, 2, 18), (4, 2, 18)}

In [23]:
n,m

(6, 8)